In [1]:
!pip install PyPortfolioOpt

pytest 4.6.6 has requirement pluggy<1.0,>=0.12, but you'll have pluggy 0.7.1 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import requests
import pandas as pd
import numpy as np
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from pandas.io.json import json_normalize
import json
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import time

In [2]:
URL = 'https://dolphin.jump-technology.com:8443/api/v1/' 
AUTH = ('ESPRIT_GROUPE2', '4Ah4XhAfmns8TnES') 
def columnsToStr(columns): 
    if isinstance(columns, list):
        k="?columns="
        for a in columns:
            k=k+a+"&columns="
        k = k[:-9]
        return k
def getData(endpointApi, date=None, full_response=False, columns=list()):
    payload = {'date': date }
    res = requests.get(URL + endpointApi + columnsToStr(columns),
    params=payload,
    auth=AUTH,
    verify=False) 
    return res.content.decode('utf-8')

In [3]:
def getRatios( ratio=[],asset=[],star_date='2013-06-13',end_date='2019-04-18', full_response=False, columns=list()):
    payload = {'ratio': ratio,'asset':asset,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
     headers={'Content-Type': 'application/x-www-form-urlencoded'},
    auth=AUTH,
    verify=False) 
    k = res.content.decode('utf-8')
    k= json.loads(k)
    kk = pd.DataFrame(columns=['ASSET_DATABASE_ID','ratio_id','ratioValue','ratioType'])
    for a in asset:
        for b in ratio:
            kk = kk.append(
                pd.DataFrame([ (a,b, str(k[str(a)][str(b)]["value"]),str(k[str(a)][str(b)]["type"]) )],columns=['ASSET_DATABASE_ID','ratio_id','ratioValue','ratioType']),
                ignore_index=True)
    return kk

def getAssets():
    col = list(["ASSET_DATABASE_ID","LABEL","TYPE","LAST_CLOSE_VALUE","CURRENCY"])
    assets = getData('asset',columns=col)
    jdata = json.loads(assets)
    assets = pd.DataFrame.from_dict(json_normalize(jdata), orient='columns')
    assets = assets[['ASSET_DATABASE_ID.value','LABEL.value','LAST_CLOSE_VALUE.value','TYPE.value',"CURRENCY.value"]]
    assets.columns =["ASSET_DATABASE_ID","LABEL","LAST_CLOSE_VALUE",'TYPE',"CURRENCY"]
    return assets



In [5]:
def getRendementAnnualisee( asset=[],ratio=[13],star_date='2013-06-14',end_date='2019-04-18', full_response=False, columns=list()):
    kk=pd.DataFrame()
    l = list()
    payload = {'ratio': ratio,'asset':asset,'benchmark':1,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
    headers={'Content-Type': 'application/x-www-form-urlencoded'},auth=AUTH,verify=False) 
    k = res.content.decode('utf-8')
    k=json.loads(k)
    for a in asset:
        l.append(str(k[str(a)]["13"]["value"]))
    kk=pd.DataFrame(l,index=asset)
    kk[kk.columns] = kk[kk.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return kk

In [6]:
def getCorrelation( asset=[],ratio=[11],star_date='2013-06-14',end_date='2019-04-18', full_response=False, columns=list()):
    kk = pd.DataFrame()
    l = list()
    for i in asset:
        payload = {'ratio': ratio,'asset':asset,'benchmark':i,'start_date': star_date,'end_date':end_date }
        res = requests.post(URL + "ratio/invoke" ,
        data=str(payload),
         headers={'Content-Type': 'application/x-www-form-urlencoded'},
        auth=AUTH,
        verify=False) 
        k = res.content.decode('utf-8')
        k=json.loads(k)
        ls=list()
        for a in asset:
            ls.append(str(k[str(a)][str(11)]["value"]))
        l.append(ls)
    kk=pd.DataFrame(l,index=asset,columns=asset)
    kk[kk.columns] = kk[kk.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return kk

In [7]:
def getVolatilite( asset=[],ratio=[10],star_date='2013-06-14',end_date='2019-04-18', full_response=False, columns=list()):
    kk=pd.DataFrame()
    l = list()
    payload = {'ratio': ratio,'asset':asset,'benchmark':1,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
    headers={'Content-Type': 'application/x-www-form-urlencoded'},auth=AUTH,verify=False) 
    k = res.content.decode('utf-8')
    k=json.loads(k)
    for a in asset:
        r=1
        l.append(str(k[str(a)]["10"]["value"]))
    kk=pd.DataFrame(l,index=asset)
    kk[kk.columns] = kk[kk.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return kk


In [8]:
def getCurrencyChange(assets):
    l = list()
    for i in assets.CURRENCY.tolist():
        result = getData('currency/rate/'+str(i)+'/to/EUR?date=2013-06-14')
        value = json.loads(result)['rate']['value'].replace(',','.')
        value = float(value)
        l.append(value)
    return l

In [9]:
def getCotations(asset_id):
    col = list(["date","asset","nav","gross","pl","close","return"])
    assetC = getData('asset/'+str(asset_id)+'/quote?start_date=2013-06-14&end_date=2019-04-18')
    jdata = json.loads(assetC)
    assetC = pd.DataFrame.from_dict(json_normalize(jdata), orient='columns')
    assetC = assetC[["asset.value","date.value","nav.value","gross.value","pl.value","close.value","return.value"]]
    assetC.columns =["asset","date","nav","gross","pl","close","return"]
    return assetC

In [10]:
def cleanData(Fasset):
    df=pd.DataFrame()
    df["asset"]=Fasset.ASSET_DATABASE_ID
    df["quantity"]=Fasset.quantity
    return df

In [11]:
def dataToJson(df):
    b=pd.DataFrame()
    b=pd.concat([b,df.asset],axis=1,sort=False)
    b=pd.concat([b,df.quantity],axis=1,sort=False)
    b=b.T
    a='{"label":"ESPRIT_PTF_2","currency":{"code": "EUR"},"type": "front","values": {"2013-06-14": ['
    for i in b.columns:
        a=a+'{"asset" :'+b[i].to_json()+'},'
    a=a[0:-1]
    a=a+"]}}"
    return a

In [12]:
def pushPortfolio(df):
    payload = dataToJson(df)
    res = requests.put(URL + "portfolio/1839/dyn_amount_compo" ,
    data=payload,
     headers={'Content-Type': 'application/x-www-form-urlencoded'},
    auth=AUTH,
    verify=False) 
    return 

In [13]:
def getSharpe( asset=[1839],ratio=[12],star_date='2013-06-14',end_date='2019-04-18', full_response=False):
    payload = {'ratio': ratio,'asset':asset,'benchmark':0,'start_date': star_date,'end_date':end_date }
    res = requests.post(URL + "ratio/invoke" ,
    data=str(payload),
    headers={'Content-Type': 'application/x-www-form-urlencoded'},auth=AUTH,verify=False) 
    k = res.content.decode('utf-8')
    k=json.loads(k)
    k=float(k[str(1839)]["12"]["value"].replace(',','.'))
    return k

In [15]:
def portfolioWeights(df):    
    start_time = time.time()
    mu = expected_returns.mean_historical_return(df)
    S = risk_models.sample_cov(df)
    # Optimise for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S, weight_bounds=(0, 0.095),gamma=1)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    return cleaned_weights

In [22]:
def cleanAssets():    
    start_time = time.time()
    assets = getAssets().dropna(0)
    actifs = assets[assets.TYPE != "INDEX"]
    actif=actifs.ASSET_DATABASE_ID.tolist();
    assetC=None
    for a in actifs.ASSET_DATABASE_ID.tolist():
        assetC2 = getCotations(a)
        assetC2 = assetC2[["date","close"]]
        assetC2 = assetC2.set_index('date')
        assetC2.columns = [str(a)]
        assetC = pd.concat([assetC,assetC2], axis=1,sort=False)
    df=assetC
    df[df.columns] = df[df.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return df

In [17]:
def getCoursAction(Fasset):
    assetC=None
    for a in Fasset.index.tolist():
        assetC2 = getCotations(a)
        assetC2 = assetC2[["date","close"]]
        assetC2 = assetC2.set_index('date')
        assetC2.columns = [str(a)]
        assetC = pd.concat([assetC,assetC2], axis=1,sort=False)
    assetC[assetC.columns] = assetC[assetC.columns].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'));
    return assetC

In [18]:
def isNan(x):
    return (x is np.nan or x != x)
def ActionValide(asset):
    l=list()
    for a in asset.columns.tolist():
        t=None
        df = asset[a].tolist()
        for i in df:
            if (isNan(i)==False):
                l.append(i)
                break
    return l

In [19]:
def getPortfolioData(df):
    a= portfolioWeights(df)
    a=pd.DataFrame.from_dict(a,orient='index')
    a.columns=["Weights"]
    b=a[a.Weights>0.0095]
    b.Weights=b.Weights/b.Weights.sum();
    b=b.sort_values(by='Weights',ascending=False);
    assets = getAssets().dropna(0)
    assets.index=assets.ASSET_DATABASE_ID
    Fasset=assets.T[b.index.tolist()].T
    Fasset["CHANGE"]=getCurrencyChange(Fasset)
    Fasset["Weights"]=b.Weights
    Fasset["CoursOC"]=ActionValide(getCoursAction(Fasset))
    Fasset["quantity"]=round(Fasset["Weights"]*100000000000/(Fasset["CHANGE"]*Fasset["CoursOC"]));
   
    return Fasset

In [20]:
df = cleanAssets()

In [21]:
t=getPortfolioData(df)

C:\Users\asmag\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [48]:
pushPortfolio(cleanData(t))